In [1]:
import torch
import torch.optim as optim
from torch.utils import data
import math
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import pickle

In [2]:


df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases'
                 + '/iris/iris.data', header=None)

df = df.sample(frac=1, random_state=0) # shuffle

df = df[df[4].isin(['Iris-virginica', 'Iris-versicolor'])] # filter

# add label indices column , i.e convert flower label to number
mapping = {k: v for v, k in enumerate(df[4].unique())}
df[5] = (2* df[4].map(mapping)) -1 # labels in {-1,1}


# normalise the data
alldata = torch.tensor(df.iloc[:, [0,1,2,3]].values, dtype=torch.float)
alldata = (alldata - alldata.mean(dim=0)) / alldata.var(dim=0)

# create datasets

target_tr = torch.tensor(df.iloc[:75, 5].values, dtype=torch.long)
target_va = torch.tensor(df.iloc[75:, 5].values, dtype=torch.long)
data_tr = alldata[:75]
data_va = alldata[75:]


In [3]:

def svm(x, w, b):
    h = (x@w.t()).sum(1) + b
    return h


In [4]:

def optimise_svm(X , y, sgd=True):
    
    dataset = data.TensorDataset(X,y) # create your datset
    dataloader = data.DataLoader(dataset, batch_size=25, shuffle=True) 
    
    w = torch.randn(1, 4, requires_grad=True)
    b = torch.randn(1, requires_grad=True)
    
    if sgd:
        opt = optim.SGD([w,b], lr=0.01, weight_decay=0.0001)
    else:
        opt = optim.Adam([w,b], lr=0.01, weight_decay=0.0001)
    
    for epoch in range(100):
        for batch in dataloader:
            opt.zero_grad()
            # YOUR CODE HERE
            output = svm(batch[0], w, b)
            loss = torch.mean(torch.clamp(1 - batch[1] * output, min=0))
            loss.backward()
            opt.step()
            
    return w,b
            
            

In [5]:
def getAccuracy(w,b, X, y):
    
      h = (X@w.t()).sum(1) + b
      predict = [1 if x > 0 else -1  for x in h.data]
      score  = accuracy_score(y, predict)
      return score
      

In [6]:

# create your dataloader
w,b = optimise_svm(data_tr, target_tr)
prediciton = svm(data_va,w,b)
loss = torch.mean(torch.clamp(1 - target_va * prediciton, min=0))
print(f"SGD loss is {loss}")
score = getAccuracy(w,b, data_tr, target_tr)
print(f"score is {score}")

SGD loss is 0.18527469038963318
score is 0.9466666666666667


In [7]:
w,b = optimise_svm(data_tr, target_tr, sgd=False)
prediciton = svm(data_va,w,b)
loss = torch.mean(torch.clamp(1 - target_va * prediciton, min=0))
print(f"Adam loss is {loss}")
score = getAccuracy(w,b, data_tr, target_tr)
print(f"score is {score}")

Adam loss is 0.16389106214046478
score is 0.9733333333333334


In [8]:

def sgd_mean(n):
    scores = np.zeros(n)
    test = 0
    for x in range(n):
        w,b = optimise_svm(data_tr, target_tr)
        #pickle.dump( w, open( "w_sgd.p", "wb" ) )
        #pickle.dump( b, open( "b_sgd.p", "wb" ) )
        #w = pickle.load( open( "w_sgd.p", "rb" ) )
        #b = pickle.load( open( "b_sgd.p", "rb" ) )
        score = getAccuracy(w,b, data_va, target_va)
        scores[x] = score
        test += score
    return scores

def adam_mean(n):
    scores = np.zeros(n)
    for x in range(n):
        w,b = optimise_svm(data_tr, target_tr, sgd=False)
        #pickle.dump( w, open( "w_adam.p", "wb" ) )
        #pickle.dump( b, open( "b_adam.p", "wb" ) )
        #w = pickle.load( open( "w_adam.p", "rb" ) )
        #b = pickle.load( open( "b_adam.p", "rb" ) )
        score = getAccuracy(w,b, data_va, target_va)
        scores[x] = score
    print(f'Adam weights are {w} ')
    return scores


In [9]:
sgd_mean_r = sgd_mean(100)
adam_mean_r = adam_mean(100)

adam_mean_arr = np.mean(adam_mean_r)
sgd_mean_arr = np.mean(sgd_mean_r)

adam_var = np.var(adam_mean_r)
sgd_var = np.var(sgd_mean_r)

print(f"sgd mean is {sgd_mean_arr} and adam mean is {adam_mean_arr}")
print(f"sgd var is {sgd_var} and adam var is {adam_var}")

Adam weights are tensor([[-0.7973,  0.2295,  0.1306, -0.5493]], requires_grad=True) 
sgd mean is 0.9083999999999999 and adam mean is 0.8868
sgd var is 0.0018014399999999994 and adam var is 0.0011537600000000006
